In [3]:
import chess
import chess.engine
from math import inf
import random


In [4]:
def evaluation_piece(piece, maximizing_player):
    try:
        int(piece)
        return 0
    except:
        var = -1
        if (piece == piece.lower() and not maximizing_player) or (piece==piece.capitalize() and maximizing_player):
            var = 1
        piece = piece.lower()
        if piece =='p':
            return 10*var #pour une possibilité de mouvement, on donne
        elif piece =='n' or piece=='b':
            return 30*var #cavalier : 8 possibilité de mvt, 8*5
                        #fou : 16 possibilité
        elif piece =='r':
            return 50*var #tour : 16 possibilité mais + 10 car plus facile de mater avec une tour
        elif piece =='q':
            return 90*var #dame : 32 possibilité + 50 (pour le mat)
        return 0
        

In [5]:
def is_win(board, tour, maximizing_player):
    if board.is_checkmate() and tour!=maximizing_player:
        return True
    return False
def is_lose(board, tour, maximizing_player):
    if board.is_checkmate() and tour==maximizing_player:
        return True
    return False
def is_draw(board):
    if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
        return True
    return False
def evaluation_plateau(board, tour, maximizing_player,max_depth, depth): 
    """
    Retourne un score au plateau en fonction du joueur dont c'est le tour (maximizing_player)
    On suppose que les minuscules dans la chaîne renvoyée par board_fen() correspondent aux noirs(False)
    board.turn = tour = True => tour des blancs
    board.turn = tour = False => tour des noirs
    """
    score=0
    if is_lose(board, tour, maximizing_player):
        return -900 * (max_depth-depth +1) #l'adversaire fait échec et mat
    elif is_win(board, tour, maximizing_player):
        return 900 * (max_depth-depth+1) #on a fait échec et mat
    elif board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
        return 0

    for elt in board.board_fen():
        if elt == '/':
            pass
        else:
            score+=evaluation_piece(elt, maximizing_player)
    return score     

In [6]:
import chess
from random import randrange
def minimax(board, tour, maximizing_player, depth=0, max_depth=3):
    """
    Renvoi le meilleur coup à jouer et le score au plateau après que ce coup soit joué
    """
    moves = [move for move in board.legal_moves]
    if board.legal_moves.count()==0 or max_depth==depth or is_win(board,tour,maximizing_player) or is_lose(board, tour, maximizing_player) or is_draw(board):
        return (evaluation_plateau(board, tour, maximizing_player, max_depth, depth), None)
    board.push(moves[0])
    score = minimax(board,not tour, maximizing_player, depth+1,max_depth)[0]
    best_move=board.pop()
    if tour==maximizing_player: #On cherche le Max
        for move in moves[1:]:
            board.push(move)
            val_board = minimax(board,not tour, maximizing_player, depth+1,max_depth)[0]
        
            if val_board>score:
                score= val_board
                best_move = board.pop()
            else:
                board.pop()
            
    else: #On cherche le Min
        for move in moves:
            board.push(move)
            val_board = minimax(board,not tour, maximizing_player, depth+1,max_depth)[0]

            if val_board<score:
                score = val_board
                best_move = board.pop()
            else:
                board.pop()
    return (score, best_move) 

In [7]:
import time
class JoueurMinimax():
    def __init__(self,board, ma_couleur):
        self.board = board
        self.color = ma_couleur=="white" or ma_couleur=="blanc"
        self.score = evaluation_plateau(board, board.turn, board.turn)
        
    def is_my_turn(self,new_board):
        if new_board.turn == self.color:
            self.board=new_board
            self.score = evaluation_plateau(board, board.turn, board.turn)
            return True
        return False
    def is_win(self):
        if self.score==inf:
            print('Victoire des: ', self.color)
            time.sleep(5)
            return True
            
        return False
    def is_lose(self):
        if self.score==-inf:
            print('Victoire des: ', not self.color)
            time.sleep(5)
            return True
        return False
    def jouer(self, new_board):
        print(self.is_my_turn(new_board), new_board.turn, self.color)
        if self.is_my_turn(new_board) and not is_lose(new_board, board.turn, self.color) and not is_draw(board):
            print("je joue")
            self.board.push(minimax(self.board)[1])
            return self.board
        return new_board



In [8]:
class JoueurAleatoire:
    def __init__(self, board, ma_couleur):
        self.board = board
        self.color = ma_couleur=="white" or ma_couleur=="blanc"
        
    def is_my_turn(self,new_board):
        if new_board.turn == self.color:
            self.board=new_board
            return True
        return False
    def is_win(self):
        if self.score==inf:
            print('Victoire des: ', self.color)
            time.sleep(5)
            return True
            
        return False
    def is_lose(self):
        if self.score==-inf:
            print('Victoire des: ', not self.color)
            time.sleep(5)
            return True
        return False
    def jouer(self, new_board):
        print(self.is_my_turn(new_board), new_board.turn, self.color)
        if self.is_my_turn(new_board) and not is_lose(new_board, board.turn, self.color) and not is_draw(board):
            moves = [move for move in board.legal_moves]
            print("je joue")
            self.board.push(random.choice(moves))
            return self.board
        return new_board

In [9]:
#board = chess.Board("6r1/1q1p4/1p4r1/2p3B1/kpP4p/1R3b1P/KP2p3/8 ")
board = chess.Board()
board.turn = True
print(board, board.turn)
print(board.legal_moves)

while not board.is_checkmate():
    board.push(minimax(board, board.turn, board.turn)[1])
    print(board, board.turn)
    print("checkmate : ",board.is_checkmate())
    print("\n")
    
    board.push(minimax(board, board.turn, board.turn)[1])
    print(board, board.turn)
    print("checkmate : ",board.is_checkmate())
    print("\n")
    print(board.move_stack,"\n", type(board.move_stack), type(board.move_stack[0]))
history = [move.san() for move in board.move_stack]

print(history)  # Affiche : ['e4', 'e5', 'Nf3']



r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R True
<LegalMoveGenerator at 0x2147fc88250 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R False
checkmate :  False


r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R True
checkmate :  False


[Move.from_uci('g1h3'), Move.from_uci('g8h6')] 
 <class 'list'> <class 'chess.Move'>
r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . N .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R False
checkmate :  False


r n b q k b . r
p p p p p p p p
. . . . . . . .
. . . . . . N .
. . . . . . n .
. . . . . . . .
P P P P P P P P
R N B Q K B . R True
checkmate :  False


[Move.from_uci('g1h3'), Move.from_uci('g8

KeyboardInterrupt: 